### Building Atoms: direct definition

ASE is Python library for _atomistic modelling_; systems are fundamentally described by a set of atomic positions, and calculations are derived from these.

When working with ASE molecules and materials are represented by the `Atoms` class. We can define a molecule with lists of symbols and positions:

In [1]:
from ase import Atoms

d = 1.10
molecule = Atoms(['N', 'N'], positions=[(0., 0., 0.), (0., 0., d)])

or, conveniently, we can compress the positions to a chemical formula:

In [2]:
molecule = Atoms('N2', positions=[(0., 0., 0.), (0., 0., d)])

It can be useful to visualise our structure to make sure it is reasonable. `ase.visualize.view` provides a simple structure viewer in a floating window; this is quite useful when working on a Python script, but can be a little bit annoying when using a Jupyter notebook.

In [3]:
from ase.visualize import view
view(molecule)

<Popen: returncode: None args: ['/home/adam/micromamba/envs/ase-tutorials/bi...>

You can spin the molecule around with right-click-and-drag, and zoom with mouse wheel.

Some alternative viewers are available for Jupyter notebooks; here we will use `nglview`. It should be pre-installed on the virtual machines for the workshop. In this viewer left-click-and-drag is used for rotation.

In [4]:
import nglview
nglview.show_ase(molecule)

NGLWidget()

Many interesting systems are crystals, described by atomic positions in a periodic unit cell. There are two relevant settings for an Atoms object: the unit cell itself and the _periodic boundary conditions_ (PBC).

In [20]:
a = 5.387
crystal = Atoms('Zn4S4',
                scaled_positions=[[0., 0., 0.],
                                  [0., 0.5, 0.5],
                                  [0.5, 0., 0.5],
                                  [0.5, 0.5, 0.],
                                  [0.25, 0.75, 0.75],
                                  [0.25, 0.25, 0.25],
                                  [0.75, 0.75, 0.25],
                                  [0.75, 0.25, 0.75]],
               cell=[a, a, a],
               pbc=True)

def show(atoms: Atoms) -> None:    
    view = nglview.show_ase(atoms)
    if any(atoms.pbc):
        view.add_unitcell()
    return view

show(crystal)

NGLWidget()

We used a few tricks to make writing in the structure a bit easier:
- The symbols were compressed to 'Zn4S4'
- Instead of working out positions in Angstrom, `scaled_positions` relative to lattice vectors were used
- The cell with just 3 values, so it is assumed to be cubic. In other cases we might use the full 3x3 matrix, e.g. `cell=[[a, 0, 0], [0, a, 0], [0, 0, a]]`
- We set `pbc=True` to indicate periodic boundary conditions in all directions. These can also be specified along each direction, e.g. `pbc=[True, True, False]` for a "slab" calculation with exposed surfaces.

We also defined a Python function `show()` which will show Atoms including a unit cell with nglview. This will save us from writing three lines of code every time!

### Examining atoms: properties and methods

Now that we have some Atoms objects we can see what information is available from them. We call some "getter" methods on the `molecule` object, which is an instance of `Atoms`.

In [6]:
print("N2 positions")
print(molecule.get_positions(), end="\n\n")

print("N2 symbols")
print(molecule.get_chemical_symbols(), end="\n\n")

print("N2 masses")
print(molecule.get_masses(), end="\n\n")

print("N2 center of mass")
print(molecule.get_center_of_mass())

N2 positions
[[0.  0.  0. ]
 [0.  0.  1.1]]

N2 symbols
['N', 'N']

N2 masses
[14.007 14.007]

N2 center of mass
[0.   0.   0.55]


The first two attributes here are not surprising; they are the information we provided. The masses were not provided: when the Atoms was created, ASE found some standard values and included them in the data set.

If we like, we can override this default and include some isotopic effects:

In [7]:
d = 1.10
isotope = Atoms('N2',
                positions=[(0., 0., 0.), (0., 0., d)],
                masses=[13.006, 14.003])

print("13N-14N masses")
print(isotope.get_masses(), end="\n\n")

print("13N-14N center of mass")
print(isotope.get_center_of_mass())


13N-14N masses
[13.006 14.003]

13N-14N center of mass
[0.         0.         0.57030249]


The center of mass was _not_ defined when the Atoms are created; it is derived from the other properties. So if we modify the masses using a _setter_, it should be recalculated correctly.

In [8]:
isotope = molecule.copy()
print("Center of mass before modifying masses:")
print(isotope.get_center_of_mass(), end='\n\n')

isotope.set_masses([13.006, 14.003])
print("Center of mass after modifying masses:")
print(isotope.get_center_of_mass())

Center of mass before modifying masses:
[0.   0.   0.55]

Center of mass after modifying masses:
[0.         0.         0.57030249]


In a jupyter notebook we can get the "docstring" of a method or function by adding `?` to the name:

In [9]:
isotope.get_center_of_mass?

And we can get access to the available methods and properties with tab-completion. In a Jupyter notebook or IPython terminal, try:

```crystal.[TAB]```

where `[TAB]` means "hit the TAB key". You should see that the Atoms object has a lot of features available!

Not all of them ll work until we start using *Calculators*, but the rest of this tutorial should include some useful ones.

#### Exercise 1:
Use tab-completion and docstrings to explore the features of Atoms. In the ZnS structure, find the distance between the first Zn atom and the four S atoms. Are some of them nearer than others?

*Hint: you may need to use the "minimum image convention"*

## Reading and writing

Even with the shortcuts above, writing out structures in Python syntax can be a bit cumbersome. There are many established file formats for this data, and ASE contains read/write functions for some of them.

For example, we might obtain a structure from an online database. Here is the [Materials Project entry for sphalerite](https://materialsproject.org/materials/mp-10695); use the "export as" button on the structure visualiser to obtain a .cif file.

In [10]:
import ase.io
from pathlib import Path

# You may need to change the path to match the location/file you downloaded
imported_crystal = ase.io.read(Path.home() / "Downloads/ZnS.cif", format='cif')

show(imported_crystal)

NGLWidget()

Using the virtual desktop, have a look at this .cif file in a text editor. CIF is quite a complicated format because it is designed to hold a lot of data relevant to crystallography. By comparison, the `FHI-aims` quantum chemistry code has a very simple structure format. Let's write the ZnS structure in this format instead:

In [11]:
ase.io.write('geometry.in', imported_crystal, scaled=True)

Taking a peek at this file:

In [12]:
%%bash
cat geometry.in

#===============================================================================
# Created using the Atomic Simulation Environment (ASE)

# Sat Apr  1 12:32:51 2023

#=======================================================
lattice_vector 5.3873657499999998 0.0000000000000000 0.0000000000000000 
lattice_vector 0.0000000000000000 5.3873657499999998 0.0000000000000000 
lattice_vector 0.0000000000000000 0.0000000000000000 5.3873657499999998 
atom_frac 0.0000000000000000 0.0000000000000000 0.0000000000000000 Zn
atom_frac 0.0000000000000000 0.5000000000000000 0.5000000000000000 Zn
atom_frac 0.5000000000000000 0.0000000000000000 0.5000000000000000 Zn
atom_frac 0.5000000000000000 0.5000000000000000 0.0000000000000000 Zn
atom_frac 0.2500000000000000 0.7500000000000000 0.7500000000000000 S
atom_frac 0.2500000000000000 0.2500000000000000 0.2500000000000000 S
atom_frac 0.7500000000000000 0.7500000000000000 0.2500000000000000 S
atom_frac 0.7500000000000000 0.2500000000000000 0.7500000000000000 S


Note that
- we didn't specify the FHI-aims format; it was correctly guessed from the filename. (This also works for ase.io.read.)
- we added a `scaled=True` option to write in fractional coordinates; by default this writer uses Cartesian coordinates.

How does this work? There are lots of relevant functions in modules under `ase.io`: `ase.io.read()` and `ase.io.write()` will automatically dispatch to these functions. If we look at `ase.io.write?` we get the signature

```python
ase.io.write(
    filename: Union[str, pathlib.PurePath, IO],
    images: Union[ase.atoms.Atoms, Sequence[ase.atoms.Atoms]],
    format: str = None,
    parallel: bool = True,
    append: bool = False,
    **kwargs: Any,
) -> None
```

`**kwargs` means "all remaining keyword arguments"; this allows extra options like `scaled=True` to be passed to writers that understand them.

More information about the supported formats can be found at https://wiki.fysik.dtu.dk/ase/ase/io/io.html and a summary can be produced from the command-line with

In [16]:
%%bash
ase info --formats

platform                 Linux-6.1.14-200.fc37.x86_64-x86_64-with-glibc2.36
python-3.10.0            /home/adam/micromamba/envs/ase-tutorials/bin/python3.10
ase-3.23.0b1-d4c62f2276  /home/adam/src/ase/ase
numpy-1.24.2             /home/adam/micromamba/envs/ase-tutorials/lib/python3.10/site-packages/numpy
scipy-1.10.1             /home/adam/micromamba/envs/ase-tutorials/lib/python3.10/site-packages/scipy
matplotlib-3.7.1         /home/adam/micromamba/envs/ase-tutorials/lib/python3.10/site-packages/matplotlib
spglib-2.0.2             /home/adam/micromamba/envs/ase-tutorials/lib/python3.10/site-packages/spglib
ase_ext                  not installed
flask-2.2.3              /home/adam/micromamba/envs/ase-tutorials/lib/python3.10/site-packages/flask
psycopg2                 not installed
pyamg                    not installed

Supported formats:
  abinit-in [rw/single]: ABINIT input file
  abinit-out [r/single]: ABINIT output file
  acemolecule-input [r/single]: ACE input file
  acemolecule

To find the extra supported `**kwargs`, look at the documentation of the lower-level functions. For example we can find the `scaled=True` option for FHI-aims at https://wiki.fysik.dtu.dk/ase/ase/io/formatoptions.html#ase.io.aims.write_aims

#### Exercise 2
Import a structure file relevant to your own research, and write it to a different format. See what keywords are available for your favourite formats; for example, VASP users are likely to be interested in using  `vasp5=True`.

### Building Atoms: more tools

The `ase.build` module [(docs)](https://wiki.fysik.dtu.dk/ase/ase/build/build.html) contains tools for building structures using parameters rather than detailed lists of positions.

#### Molecules
Definitions for a set of simple molecules (the "G2" set, plus a few extra) are included with ASE. So in fact the easiest way to get an N2 molecule is

In [21]:
import ase.build
g2_n2 = ase.build.molecule('N2')
show(g2_n2)

NGLWidget()

And it's just as easy to get a buckyball! This feature is very useful for testing and trying things out.

In [22]:
show(ase.build.molecule('C60'))

NGLWidget()

#### Crystals

The equivalent tool for crystals is `ase.build.bulk`. This includes lattice parameters for some elemental reference states (the list is [in the code here](https://gitlab.com/ase/ase/-/blob/6ac638d0c699f7bc80c10a8dccb7d42eda011be2/ase/data/__init__.py#L578)),
but we can also use known lattice parameters to build structures. So we get copper "for free":

In [24]:
show(ase.build.bulk('Cu', cubic=True))

NGLWidget()

but for ZnS we have to provide a bit more information:

In [26]:
show(
    ase.build.bulk('ZnS',
                   crystalstructure='zincblende',
                   a=5.387,
                   cubic=True)
)

NGLWidget()